In [2]:
import requests
from pprint import pprint
from config import token
import requests
import datetime
import numpy as np
import pandas as pd
import os
import sys
import time

In [180]:
# get the location list
tk = {'token': token}

# search city
params="locationcategoryid=CITY&sortfield=name&sortorder=desc&limit=1000&offset="
base_url= "https://www.ncdc.noaa.gov/cdo-web/api/v2/locations"
offset=1


city_list=[]
while True:
    cur_params = params + str(offset)
    r = requests.get(base_url, params = cur_params, headers=tk)
    location_list = r.json() 
    if len(location_list) == 0:
        break
    city_name = []
    for i in location_list['results']:
        if "CITY:US" not in i['id']:
            continue
        city_name.append([i['id'], i['name']])
    offset += 1000
    city_list.extend(city_name)
    
# all US cities from noaa   
df = pd.DataFrame.from_dict(city_list)
df.info()
df.to_csv("all_cities.csv")
    
# shared cities from both Employment and Temperature datasets
city_list_msa = pd.read_csv("MSA.csv")[["Unified State", "Unified City"]]
shared_cities=[]
diff_cities=[]
for s1, c1 in city_list_msa.values.tolist():
    found = 0
    for id, cname in city_list:
        c2 = cname.split(',')[0]
        s2 = cname.split(' ')[-2]
        if s1 == s2 and c1 == c2:
            shared_cities.append([id, c2, s2])
            found = 1
            break
    if found != 1:
        diff_cities.append([s1, c1])
    
df = pd.DataFrame.from_dict(shared_cities)
df.info()
df.to_csv("shared_cities.csv")
df = pd.DataFrame.from_dict(diff_cities)
df.info()
df.to_csv("diff_cities.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845 entries, 0 to 844
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       845 non-null    object
 1   1       845 non-null    object
dtypes: object(2)
memory usage: 13.3+ KB
Salt Lake City
Salt Lake City
UT
UT
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       348 non-null    object
 1   1       348 non-null    object
 2   2       348 non-null    object
dtypes: object(3)
memory usage: 8.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       41 non-null     object
 1   1       41 non-null     object
dtypes: object(2)
memory usage: 784.0+ bytes


In [241]:
# search weather
params = 'datasetid=GSOM'
date_range = '&datatypeid=TMAX&datatypeid=TMIN&datatypeid=TAVG&startdate=2019-01-01&enddate=2019-12-31&units=standard&limit=1000&offset='
base_url= "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

temp_city={}
for id, city, state in shared_cities:
    cur_params = params + "&locationid=" + id
    cur_params = cur_params + date_range
    key = city + ", " + state
    data = []
    offset = 1
    while True:
        cur_params = cur_params + str(offset)
        r = requests.get(base_url, params = cur_params, headers=tk)
        data_list = r.json() 
        if len(data_list) == 0:
            break
        offset += 1000
        temp = [[i['date'], i['station'], i['datatype'], i['value']] for i in data_list['results']] 
        data.extend(temp)
    temp_city[key] = data
    time.sleep(0.2)

In [253]:
#output json file
import json
with open("temp_per_city.json", 'w') as outfile:
    json.dump(temp_city, outfile)

In [29]:
# load from json and test one city
import json
with open("temp_per_city.json", 'r') as infile:
    tc = json.load(infile)

pprint(tc["Gadsden, AL"])

[['2019-01-01T00:00:00', 'GHCND:USC00013154', 'TAVG', 44.8],
 ['2019-01-01T00:00:00', 'GHCND:USC00013154', 'TMAX', 53.9],
 ['2019-01-01T00:00:00', 'GHCND:USC00013154', 'TMIN', 35.7],
 ['2019-02-01T00:00:00', 'GHCND:USC00013154', 'TAVG', 51.9],
 ['2019-02-01T00:00:00', 'GHCND:USC00013154', 'TMAX', 59.8],
 ['2019-02-01T00:00:00', 'GHCND:USC00013154', 'TMIN', 44.0],
 ['2019-03-01T00:00:00', 'GHCND:USC00013154', 'TAVG', 53.4],
 ['2019-03-01T00:00:00', 'GHCND:USC00013154', 'TMAX', 63.2],
 ['2019-03-01T00:00:00', 'GHCND:USC00013154', 'TMIN', 43.6],
 ['2019-04-01T00:00:00', 'GHCND:USC00013154', 'TAVG', 64.6],
 ['2019-04-01T00:00:00', 'GHCND:USC00013154', 'TMAX', 75.8],
 ['2019-04-01T00:00:00', 'GHCND:USC00013154', 'TMIN', 53.4],
 ['2019-05-01T00:00:00', 'GHCND:USC00013154', 'TAVG', 73.9],
 ['2019-05-01T00:00:00', 'GHCND:USC00013154', 'TMAX', 83.6],
 ['2019-05-01T00:00:00', 'GHCND:USC00013154', 'TMIN', 64.2],
 ['2019-06-01T00:00:00', 'GHCND:USC00013154', 'TAVG', 77.4],
 ['2019-06-01T00:00:00',

In [30]:
# get the average temperature for each month at a specific city
import csv
month=["2019-01-01", "2019-02-01", "2019-03-01","2019-04-01","2019-05-01","2019-06-01",\
       "2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01"]

with open("aver_temp_per_city.csv", 'w') as outfile:
    csvwriter = csv.writer(outfile)
    csvwriter.writerow(['city', 'Jan', 'Feb', 'Mar', 'Apr','May', 'Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec'])
    rows=[]
    for city in tc: # for each city
        aver_per_month = [0]*12 # each city has 12 data
        count_per_month = [0]*12 # each data is average of all TAVG from stations
        for record in tc[city]: # each city's json 
            for m_index in range(0, len(month)): # m_index represents each month
                if month[m_index] in record[0]: # if month belongs to 0 under each city's json
                    aver_per_month[m_index] += float(record[3]) # add this ave to previous sum
                    count_per_month[m_index] += 1 # count 1 to previous count_sum
                    break # break the inner for loop if found the month info
        #city, sum and count are available now
        city_data = [city] # assign city row into the file
        for i in range(0, len(aver_per_month)): # assign each month's average temp to sequential 12 columns
            if count_per_month[i] == 0:
                print(city + " has no record for " + month[i]) #figure out missing avg
                city_data.append('N/A')
                continue
            city_data.append('{:.2f}'.format(aver_per_month[i] / count_per_month[i]))# formating
        rows.append(city_data)
    csvwriter.writerows(rows)

Gadsden, AL has no record for 2019-10-01
Gadsden, AL has no record for 2019-12-01
Chico, CA has no record for 2019-12-01
Ocala, FL has no record for 2019-10-01
Elizabethtown, KY has no record for 2019-01-01
Elizabethtown, KY has no record for 2019-02-01
Elizabethtown, KY has no record for 2019-03-01
Elizabethtown, KY has no record for 2019-04-01
Elizabethtown, KY has no record for 2019-05-01
Elizabethtown, KY has no record for 2019-06-01
Elizabethtown, KY has no record for 2019-07-01
Elizabethtown, KY has no record for 2019-08-01
Elizabethtown, KY has no record for 2019-09-01
Elizabethtown, KY has no record for 2019-10-01
Elizabethtown, KY has no record for 2019-11-01
Elizabethtown, KY has no record for 2019-12-01
Owensboro, KY has no record for 2019-01-01
Owensboro, KY has no record for 2019-02-01
Owensboro, KY has no record for 2019-03-01
Owensboro, KY has no record for 2019-04-01
Owensboro, KY has no record for 2019-05-01
Owensboro, KY has no record for 2019-06-01
Owensboro, KY has 